# Project #2 - ETL Boogaloo
## Washington University/Trilogy Data Science Winter 2020
### James Avery / Robert Hanlon / Michael Joyce

For our second project we were given free reign to choose our own data sources.  We wanted to choose something that is culturally relevant to now, and since marijuana legalization just took effect in Missouri in January, something along those lines sounded interesting.

Our research into datasets relating to legal marijuana led us to the [Colorado state government website](https://data.colorado.gov/), where they have a very nice interface and data going back to 2014, shortly after the dawn of recreational legalization in their state.

In [5]:
import pandas as pd

In [6]:
#requires:
# pip install boto3
# pip install s3fs <-- Pandas uses this with .to_csv for (I assume) serializing the dataframe to csv.
import boto3
from io import StringIO

### The user must enter the AWS S3 credentials for this bucket.  Contact me (Robert Hanlon, rhanlon70@gmail.com) for those credentials.

#### This was done in lieu of hardcoding credentials for a few reasons.
1. AWS actually called me (and emailed me) and told me to change the keys almost immediately after I first committed them to Github (forgetting that Github was totally open to the public).  Not just the keys I committed, but all the keys that were in existence at the time I committed the one set.
1. The credentials may change, and I didn't want to have to change the code each time they did.
1. Hardcoding credentials is worst practices.

In [8]:
aws_access_key_id = input("Enter the AWS Access Key ID:")

In [10]:
aws_secret_access_key = input("Enter the AWS Secret Access Key:")

### Now we create the AWS S3 proxy object, and verify access to the specific bucket that will contain all the data.

In [11]:
s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [12]:
bucket_name = 'wustl-data-01-2020-project-2'
bucket_path = f's3://{bucket_name}/dfs'
bucket_path

's3://wustl-data-01-2020-project-2/dfs'

In [13]:
# validate that this bucket is accessible.  If not, expect an exception to be thrown here.
bucket = s3.Bucket(bucket_name)
bucket

s3.Bucket(name='wustl-data-01-2020-project-2')

In [14]:
#Loaded the new Licensed MJ Businesses CSV into a dataframe

license = pd.read_csv('Licensed_MJ_Businesses_fixed_lic_no.csv')
license.head()

,Licensee,DBA,License_No,Street_Address,City,ZIP,Category,Month,Year,Certification
0,1617 WAZEE STREET LLC,NaN,NaN,1617 Wazee St,Denver,80202.0,Medical Marijuana Centers,1,2013,NaN
1,3-D DENVER'S DISCREET DISPENSARY LLC,NaN,NaN,4305 Brighton Blvd,Brighton,80216.0,Medical Marijuana Centers,1,2013,NaN
2,"4625 E. COLFAX, LLC",NaN,NaN,4625 E. Colfax Ave,Denver,80216.0,Medical Marijuana Centers,1,2013,NaN
3,5280 WELLNESS LLC,NaN,NaN,1321 Elati Street,Denver,80204.0,Medical Marijuana Centers,1,2013,NaN
4,A CUT OFF THE TOP MEDICAL MARIJUANA,NaN,NaN,2059 W. 9th Ave,Denver,80204.0,Medical Marijuana Centers,1,2013,NaN


# clean license dataframe

In [15]:
#removed some columns 
cleaned_license=license.drop(['Certification','Month','ZIP','DBA'],axis=1)

In [16]:
#removed null values
cleaned_license=cleaned_license.dropna(subset=['Year','Category','City','Street_Address','License_No','Licensee'])

In [17]:
cleaned_license.head()

,Licensee,License_No,Street_Address,City,Category,Year
6308,1617 WAZEE STREET LLC,402-00017,"1617 Wazee Street, Unit B",Denver,Medical Marijuana Centers,2014
6309,3-D DENVER'S DISCREET DISPENSARY LLC,402-00078,4305 Brighton Boulevard,Denver,Medical Marijuana Centers,2014
6310,4 REAL CANNABIS SUPPLY LLC,402-00471,County Rd 17 Hwy 160,Ft Garland,Medical Marijuana Centers,2014
6311,505 CENTER LLC,402-00503,505 West 40th Avenue,Denver,Medical Marijuana Centers,2014
6312,5110 RACE LLC,402-00204,5110 Race Street,Denver,Medical Marijuana Centers,2014


In [18]:
#license.to_csv(f'{bucket_path}/license.csv', encoding='utf-8', index=False)
#license.to_csv('license.csv', encoding='utf-8', index=False)
csv_buffer = StringIO()
license.to_csv(csv_buffer)
s3.Object(bucket_name, 'dfs/license.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '3010BF5B8162BC09',
  'HostId': 'RVFdf/ALIJ7/rf0iiwfjttHTEWi1y513z8XfetEJ7eXyGhyXEzwhwEGxEUDhDzOYKj/MWMuCYAs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'RVFdf/ALIJ7/rf0iiwfjttHTEWi1y513z8XfetEJ7eXyGhyXEzwhwEGxEUDhDzOYKj/MWMuCYAs=',
   'x-amz-request-id': '3010BF5B8162BC09',
   'date': 'Sat, 25 Apr 2020 22:22:08 GMT',
   'etag': '"7928047c99b92df02a76b8e9f0adc7d5"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"7928047c99b92df02a76b8e9f0adc7d5"'}

In [19]:
sales_revenue=pd.read_csv('Marijuana_Sales_Revenue_in_Colorado.csv')
sales_revenue.head()

,Month,Year,County,Med_Sales,Rec_Sales,Med_Blank_Code,Rec_Blank_Code,ID
0,12,2019,Yuma,NaN,NaN,NL,NL,122019Yuma
1,12,2019,Weld,314840.0,2448689.0,NaN,NaN,122019Weld
2,12,2019,Washington,NaN,NaN,NL,NL,122019Washington
3,12,2019,Teller,NaN,NaN,NL,NL,122019Teller
4,12,2019,Summit,NaN,2476854.0,NR,NaN,122019Summit


In [20]:
#Dropped boring columns from Sales CSV

simple_sales = sales_revenue.drop(columns=['Med_Blank_Code', 'Rec_Blank_Code'])
simple_sales.head()

,Month,Year,County,Med_Sales,Rec_Sales,ID
0,12,2019,Yuma,NaN,NaN,122019Yuma
1,12,2019,Weld,314840.0,2448689.0,122019Weld
2,12,2019,Washington,NaN,NaN,122019Washington
3,12,2019,Teller,NaN,NaN,122019Teller
4,12,2019,Summit,NaN,2476854.0,122019Summit


In [21]:
#Dropped all rows with NaN values in both Med_Sales and Rec_Sales

clean_sales = simple_sales.dropna(how="all", subset=["Rec_Sales","Med_Sales"])
clean_sales.head()

,Month,Year,County,Med_Sales,Rec_Sales,ID
1,12,2019,Weld,314840.0,2448689.0,122019Weld
4,12,2019,Summit,NaN,2476854.0,122019Summit
5,12,2019,Sum of NR Counties,27314662.0,116437714.0,122019Sum of NR Counties
7,12,2019,San Miguel,NaN,444533.0,122019San Miguel
8,12,2019,San Juan,NaN,13232.0,122019San Juan


In [22]:
#Loaded the new County Statistics CSV into a dataframe

med_mj_stats=pd.read_csv('MM_County_Statistics_full_percts.csv')
med_mj_stats.head()

,County,Number of Patients,Percent of Patients
0,Adams,8772,7.90
1,Alamosa,204,0.18
2,Arapahoe,11985,10.79
3,Archuleta,378,0.34
4,Baca,35,0.03


In [23]:
med_mj_stats.dtypes

County                  object
Number of Patients       int64
Percent of Patients    float64
dtype: object

In [24]:
tax_revenue=pd.read_csv('State_Sales_Tax_Revenue_from_Marijuana_in_Colorado.csv')
tax_revenue.head()

,Month,Year,County,Med_Tax_Rev,Rec_Tax_Rev,Med_RemainderOfState_Counties,Rec_RemainderOfState_Counties,Med_Blank_Code,Rec_Blank_Code,ID
0,8,2019,Adams,17037.0,NaN,NaN,NaN,NaN,NaN,82019Adams
1,8,2019,Alamosa,NaN,NaN,NaN,NaN,NR,NaN,82019Alamosa
2,8,2019,Arapahoe,24256.0,NaN,NaN,NaN,NaN,NaN,82019Arapahoe
3,8,2019,Boulder,33557.0,NaN,NaN,NaN,NaN,NaN,82019Boulder
4,8,2019,Chaffee,NaN,NaN,NaN,NaN,NR,NaN,82019Chaffee


In [25]:
#Dropped boring columns

simple_taxes = tax_revenue.drop(columns=["Med_RemainderOfState_Counties", "Rec_RemainderOfState_Counties", "Med_Blank_Code", "Rec_Blank_Code"])
simple_taxes.head()

,Month,Year,County,Med_Tax_Rev,Rec_Tax_Rev,ID
0,8,2019,Adams,17037.0,NaN,82019Adams
1,8,2019,Alamosa,NaN,NaN,82019Alamosa
2,8,2019,Arapahoe,24256.0,NaN,82019Arapahoe
3,8,2019,Boulder,33557.0,NaN,82019Boulder
4,8,2019,Chaffee,NaN,NaN,82019Chaffee


In [26]:
#Dropped all rows with NaN in both Med_Tax_Rev and Rec_Tax_Rev

clean_taxes = simple_taxes.dropna(how="all", subset=["Med_Tax_Rev","Rec_Tax_Rev"])
clean_taxes.head()

,Month,Year,County,Med_Tax_Rev,Rec_Tax_Rev,ID
0,8,2019,Adams,17037.0,NaN,82019Adams
2,8,2019,Arapahoe,24256.0,NaN,82019Arapahoe
3,8,2019,Boulder,33557.0,NaN,82019Boulder
6,8,2019,Denver,392226.0,NaN,82019Denver
7,8,2019,Eagle,2826.0,NaN,82019Eagle


In [27]:
zipcty = pd.read_csv('ZIP-COUNTY-FIPS_2018-03.csv')
zipcty.head()

,ZIP,STCOUNTYFP,CITY,STATE,COUNTYNAME,CLASSFP
0,36091,1001,Verbena,AL,Autauga County,H1
1,36758,1001,Plantersville,AL,Autauga County,H1
2,36006,1001,Billingsley,AL,Autauga County,H1
3,36067,1001,Prattville,AL,Autauga County,H1
4,36701,1001,Selma,AL,Autauga County,H1


##  This dataframe contains information for the entire country, but the other dataframes only contain data from Colorado, so we can cut this dataframe down to approximately 1/50 of its size.

In [28]:
co_only = zipcty.loc[zipcty['STATE'] == 'CO']
co_only.head()

,ZIP,STCOUNTYFP,CITY,STATE,COUNTYNAME,CLASSFP
5549,80229,8001,Denver,CO,Adams County,H1
5550,80701,8001,Fort morgan,CO,Adams County,H1
5551,80642,8001,Hudson,CO,Adams County,H1
5552,80654,8001,Wiggins,CO,Adams County,H1
5553,80247,8001,Denver,CO,Adams County,H1


# The Load process
We decided that the output to be persisted into the S3 bucket should be a CSV file, rather than uploading the Pandas Python object itself, so that what's in the bucket is a human-readable artifact, and didn't need to be deserialized back into a Pandas dataframe object in order to see the data.

Four of the datasets were persisted directly from their Pandas dataframes to S3.  A fifth, MM_County_Statistics_full_percts, was exported as a CSV to the local filesystem, where additional transformation was done via Excel.  When that was complete, that finished CSV file was uploaded into the S3 bucket via manual drag-and-drop.

## Some notes on the AWS persistence mechanism
A few variations were attempted in order to get the data out of the Pandas dataframe and into the S3 bucket as a CSV file.  Passing the bucket_path ("s3://aws...") as the first argument of the dataframe's to_csv() method was syntactically acceptable, but I kept encountering encoding errors -- a \ufffd Unicode character, specifically.  It didn't matter whether I explicitly set the encoding to utf-8 when reading the data in from the source CSV's, or set the encoding to utf-8 in the same .to_csv() method, which leads me to believe that it's some quirk of the internal workings of Pandas itself.

The solution, [gleaned from Stack Overflow](https://stackoverflow.com/questions/38154040/save-dataframe-to-csv-directly-to-s3-python), was to use a buffer StringIO object from the Python io package.  Combining that, the Pandas to_csv() method, and the boto3 s3 proxy object, we were able to successfully persist the dataframe data to S3 as a CSV.

In [29]:
csv_buffer = StringIO()
co_only.to_csv(csv_buffer)
s3.Object(bucket_name, 'dfs/co_only.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'FE7EB2B192BD3A14',
  'HostId': 'JHfVZNIZERQyE5sM542jmIHUM7h+IETkrf7R/KoI3pKEuqaKlMssbZx2F/mP/L5oYnCRXbQjAA8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'JHfVZNIZERQyE5sM542jmIHUM7h+IETkrf7R/KoI3pKEuqaKlMssbZx2F/mP/L5oYnCRXbQjAA8=',
   'x-amz-request-id': 'FE7EB2B192BD3A14',
   'date': 'Sat, 25 Apr 2020 22:23:04 GMT',
   'etag': '"81b4adb7d1caba48ee239790c4b6ab16"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"81b4adb7d1caba48ee239790c4b6ab16"'}

In [30]:
csv_buffer = StringIO()
clean_sales.to_csv(csv_buffer)
s3.Object(bucket_name, 'dfs/clean_sales.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '988FAB23F56C465A',
  'HostId': 'kuhFhrlzrvwOzcdcx1hKwRzHEyyjQyqhQGElD48LTC92lxecoySjvUrd+XgEFwKnB3sp06XvJKM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'kuhFhrlzrvwOzcdcx1hKwRzHEyyjQyqhQGElD48LTC92lxecoySjvUrd+XgEFwKnB3sp06XvJKM=',
   'x-amz-request-id': '988FAB23F56C465A',
   'date': 'Sat, 25 Apr 2020 22:23:08 GMT',
   'etag': '"0d5bd829900d77208961d7ae87a2a5ac"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"0d5bd829900d77208961d7ae87a2a5ac"'}

In [31]:
csv_buffer = StringIO()
clean_taxes.to_csv(csv_buffer)
s3.Object(bucket_name, 'dfs/clean_taxes.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '3C82A9AACF4CF6E7',
  'HostId': '7mHnNnLWOLRj/ZceVRbozxJjm0YdG+N32TJcLxQydkGGObVvyg3Zaokk1YnQTgRbKoWBv4/qngA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7mHnNnLWOLRj/ZceVRbozxJjm0YdG+N32TJcLxQydkGGObVvyg3Zaokk1YnQTgRbKoWBv4/qngA=',
   'x-amz-request-id': '3C82A9AACF4CF6E7',
   'date': 'Sat, 25 Apr 2020 22:23:10 GMT',
   'etag': '"46e3e2d5c1038de59c78f861407a7dd5"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"46e3e2d5c1038de59c78f861407a7dd5"'}